In [ ]:
import torch
import json
import torchvision
from torchvision import transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader, Subset, ConcatDataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.utils import save_image
from statistics import mean
import numpy as np
import random
import math
import sys
import time
from PIL import Image
import itertools
from fedlab.utils.dataset.partition import MNISTPartitioner
from buyer-insight_algorithm import*
from algorithm_greedy import*
from algorithm_exploration import*
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
class FlippedLabelDataset(Dataset):
    def __init__(self, dataset, flip_fraction=0.75):
        self.dataset = dataset
        self.flip_fraction = flip_fraction

    def __getitem__(self, index):
        data, label = self.dataset[index]
        # Flip the label with a probability of flip_fraction
        if np.random.rand() < self.flip_fraction:
            # Example of flipping: label -> 9 - label
            label = 9 - label
        return data, label

    def __len__(self):
        return len(self.dataset)

In [ ]:
def modify_dataloaders(data_loaders, flip_fraction=0.75, indexes):
    new_data_loaders = {}
    i = 1
    for partition_name, loader in data_loaders.items():
        if i in indexes:
            flipped_dataset = FlippedLabelDataset(loader.dataset, flip_fraction)
            new_loader = DataLoader(flipped_dataset, batch_size=loader.batch_size, shuffle=True, num_workers=loader.num_workers)
            new_data_loaders[partition_name] = new_loader
        else:
            new_data_loaders[partition_name] = loader
        i += 1
    return new_data_loaders

In [ ]:
class CombinedDataset(Dataset):
    def __init__(self, dataset1, dataset2):
        self.dataset1 = dataset1
        self.dataset2 = dataset2
        self.length1 = len(dataset1)
        self.targets = dataset1.targets + dataset2.targets

    def __len__(self):
        return len(self.dataset1) + len(self.dataset2)

    def __getitem__(self, idx):
        if idx < self.length1:
            return self.dataset1[idx]
        else:
            return self.dataset2[idx - self.length1]

In [ ]:
def create_combined_loaders(data_loaders, catalogue, original_lengths):
    original_keys = list(data_loaders.keys())
    combined_loaders = {}
    combined_lengths = []  # List to store the number of data points in each combined DataLoader

    # Start by adding the original loaders and their lengths
    for key, length in zip(original_keys, original_lengths):
        combined_loaders[key] = data_loaders[key]
        combined_lengths.append(length)  # Add the length of the original DataLoader

    # Generate combinations of increasing size
    i = len(original_keys) + 1  # Start naming combined loaders from here
    for n in range(2, len(original_keys) + 1):
        for combo in itertools.combinations(original_keys, n):
            if len(combined_loaders) >= catalogue:
                return combined_loaders, combined_lengths  # Return early if catalogue limit reached
            # Sum the lengths of the datasets in the current combination
            combo_length = sum(original_lengths[original_keys.index(k)] for k in combo)
            combined_datasets = ConcatDataset([data_loaders[k].dataset for k in combo])
            # Assuming all data loaders use the same batch size
            batch_size = data_loaders[original_keys[0]].batch_size
            combined_loaders[i-1] = DataLoader(combined_datasets, batch_size=batch_size, shuffle=True)
            combined_lengths.append(combo_length)  # Append the combined length to the list
            i += 1

    return combined_loaders, combined_lengths


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2)  # 16 filters
        self.pool = nn.MaxPool2d(2, 2)  # 2x2 max pooling
        self.fc1 = nn.Linear(16 * 14 * 14, 10)  # Fully connected layer

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Convolution -> ReLU -> Pooling
        x = x.view(-1, 16 * 14 * 14)  # Flatten the tensor for the fully connected layer
        x = self.fc1(x)  # Fully connected layer
        return F.log_softmax(x, dim=1)

In [ ]:
def fetch_quality(loader, batch_size, model):
    reset_all_weights(model)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss().to(device)
    
    num_epochs = 20
    for epoch in range(num_epochs):
        model.train()
        for batch_idx, (data, targets) in enumerate(loader):
            data, targets = data.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        
            #if (batch_idx + 1) % 100 == 0:
                #print(f"Epoch [{epoch + 1}/{num_epochs}], Batch [{batch_idx + 1}/{len(loader)}], Loss: {loss.item():.4f}")
            
        #train_accuracy = calculate_accuracy(model, loader)
        #print(f"Epoch [{epoch + 1}/{num_epochs}] - Training Accuracy: {train_accuracy:.2f}%")
        test_accuracy = calculate_accuracy(model, test_loader)
    
    #print("Training and testing completed @--> ",test_accuracy)
    return test_accuracy

In [ ]:
def calculate_accuracy(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, targets in test_loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
    return (correct / total) * 100.0


In [ ]:
def measure_loader(loader):
    num_batches = len(loader)
    batch_size = loader.batch_size
    total_images = num_batches * batch_size

    for _, last_batch in enumerate(loader, 1):
        pass

    last_batch_size = len(last_batch[0])

    if last_batch_size < batch_size:
        total_images = total_images - batch_size +last_batch_size

    return total_images

In [ ]:
batch_size = 125
size_of_V = 10 
seed = 2024
indexes = [] #List the indexes of the datasets to degrade by label-flipping

    
transform = transforms.Compose([transforms.ToTensor()])

trainset = torchvision.datasets.MNIST(root="/data/MNIST/", train=True, download=True, transform=transform)
    
testset = torchvision.datasets.MNIST(root="/data/MNIST/", train=False, download=True, transform=transform)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False)
    
hetero_dir_part = MNISTPartitioner(trainset.targets, size_of_V, balance=None, partition="dirichlet", dir_alpha=0.3, verbose=False, seed=seed)

images_per_seller = np.zeros(size_of_V)
for i in range(size_of_V):
    images_per_seller[i] = len(hetero_dir_part[i])

with open(f"MNIST_len.json", 'w') as f:
    json.dump(images_per_seller.tolist(), f)
    
avg_images = np.mean(images_per_seller)
    
data_loaders = {}
partition_indices = hetero_dir_part.client_dict
    
for partition_name, indices in partition_indices.items():
    partition = Subset(trainset, indices)
    data_loader = DataLoader(partition, batch_size=batch_size, shuffle = True)
    data_loaders[partition_name] = data_loader
      

if indexes:
    complete_data_loaders = modify_dataloaders(data_loaders, 0.75, indexes)
else:
    complete_data_loaders = data_loaders
    
catalogue = (2**size_of_V)-1  # Desired number of datasets
new_data_loaders, lens = create_combined_loaders(complete_data_loaders, catalogue, images_per_seller)     
results = np.zeros((2, catalogue))


i = 0
for loader_name, train_loader in new_data_loaders.items():
    print(f"Training on loader: {i}")
    model = CNN().to(device)
    start_time = time.time()
    results[0,i] = fetch_quality(train_loader, batch_size, model)
    results[1,i] = time.time() - start_time
    print(f"Accuracy with {i}: {results[0,i]}%, took: {results[1,i]}s")
    with open(f"MNIST_FM.json", 'w') as f:
        json.dump(results.tolist(), f)
    i += 1

In [ ]:
# Generate a set of keys for the combinations in the catalog
def generate_keys(catalogue_size, size_of_V):
    # Generate the original keys (assuming they are simply integers starting from 1 for simplicity)
    original_keys = list(range(1, size_of_V + 1))

   # Initialize the list to store the keys representing combinations of original datasets
    combination_keys_as_tuples = []

    # Start by adding the original datasets' keys as single-element tuples
    for key in original_keys:
        combination_keys_as_tuples.append((key,))

    # Generate combinations of increasing size and create keys representing these combinations as tuples
    for n in range(2, len(original_keys) + 1):
        for combo in itertools.combinations(original_keys, n):
            if len(combination_keys_as_tuples) >= catalogue_size:
                # Break early if catalogue limit is reached or about to be reached
                break
            # Create a tuple from the combo, which already contains integers
            combo_tuple = tuple(combo)
            combination_keys_as_tuples.append(combo_tuple)

    # Check if we reached the limit or need to stop
    if len(combination_keys_as_tuples) > catalogue_size:
        # Trim the list to the catalogue size if it exceeded the limit
        combination_keys_as_tuples = combination_keys_as_tuples[:catalogue_size]

    single_key_combinations = [t for t in combination_keys_as_tuples if len(t) == 1]
    multiple_key_combinations = [t for t in combination_keys_as_tuples if len(t) > 1]
    if (size_of_V,) in multiple_key_combinations:
        multiple_key_combinations.remove((size_of_V,))  # Remove from multiple-key list
        single_key_combinations.append((size_of_V,))  # Add to single-key list

    return single_key_combinations, multiple_key_combinations, combination_keys_as_tuples

In [ ]:
def generate_prices(accuracies, pricing_methoK, price_per_image, correlation, catalogue, n, images_per_seller, len(indexes)):
    keys_K, keys_U, keys = generate_keys(catalogue, n)
    prices = pricing_function(accuracies[:n], pricing_methoK, price_per_image, correlation, images_per_seller, len(indexes))
    combination_prices_list = []

    # Loop over the multiple-key tuples to calculate and store their prices in the list
    for combo in keys_U:
        # Sum the prices of the components of the combination
        combo_price = sum(prices[key - 1] for key in combo)  # Adjust index for 0-based indexing
        combination_prices_list.append(combo_price)
    
    full_prices = np.hstack((np.array(prices), np.array(combination_prices_list)))
    
    return full_prices

In [ ]:
def pricing_function(qualities, pricing_methoK, price_per_image, correlation,images_per_seller):
    if pricing_method == "random":
        return np.random.rand(len(qualities))
    
    if pricing_method == "volume":
        return images_per_seller[:size_of_V] * price_per_image
    
    if pricing_method == "correlated":
        x1 = np.random.rand(len(qualities))
        return  (correlation * qualities + np.sqrt(1-correlation**2)*x1)
    
    if pricing_method == "market_based":
        prices = []
        for i in range(size_of_V):
            if images_per_seller[i]<np.mean(images_per_seller[:size_of_V]) and qualities[i] >= np.mean(qualities[:size_of_V]):
                prices.append(images_per_seller[i] * np.random.lognormal(0.617452372, 0.657475287))
            else: 
                prices.append(images_per_seller[i] * np.random.lognormal(-0.714563084, 0.342099957))
    
        return prices

In [ ]:
def create_U_and_K_sets(original_array, size_of_V, size_of_K, keys):
    num_cols = original_array.shape[1]
    assert size_of_K <= size_of_V "size_of_K should not be larger than size_of_V"
    
    if size_of_K == size_of_V:
        # If N is 10, D includes all first 10 columns, and U includes the rest
        K = original_array[:, :size_of_V]
        U = original_array[:, size_of_V:]
        keys_K = keys[:size_of_V]
        keys_U = keys[size_of_V:]
    else:
        # Randomly selecting N unique column indices from the first size_of_V columns
        random_col_indices = np.random.choice(size_of_V, size=size_of_K, replace=False)
        # Creating D with the randomly selected columns
        K = original_array[:, random_col_indices]
        # Creating keys_d with the randomly selected keys
        keys_K = [keys[i] for i in random_col_indices]
        
        # Creating a mask for all columns not included in K
        mask = np.ones(num_cols, dtype=bool)
        # Set all selected columns in the first size_of_V to False in the mask
        mask[random_col_indices] = False
        # Ensure that the rest of the first size_of_V columns are correctly identified
        for index in range(size_of_V):
            if index not in random_col_indices:
                mask[index] = True
        # Creating U with the remaining columns
        U = original_array[:, mask]
        # Creating keys_u with the remaining keys
        keys_U = [keys[i] for i in range(num_cols) if mask[i]]

    return K, U, keys_K, keys_U

In [ ]:
def scale_sets(K, U, budget_scale, keys_K, keys_U):
    # Calculate the budget ceiling based on the maximum of the second row in U
    B = U[1,:]*budget_scale
    
    # Identify columns in U where the second row does not exceed B
    valid_indices_u = np.where(U[1] <= B)[0]
   
    # Filter U and keys_u accordingly
    U = U[:, valid_indices_u]
    keys_U = [keys_U[i] for i in valid_indices_u]  # Ensure this list comprehension does not go out of range

    # Sort D by the first row in descending order and update keys_d
    sorted_indices_k = np.argsort(-K[0])
    K = K[:, sorted_indices_k]
    keys_K = [keys_K[i] for i in sorted_indices_k]

    # Sort U by the second row in increasing order and update keys_u
    sorted_indices_u = np.argsort(U[1])
    U = U[:, sorted_indices_u]
    keys_U = [keys_U[i] for i in sorted_indices_u]
    return K, U, keys_K, keys_U, B

In [ ]:
def filter_combinations(data, keys, target_key, exploration_depth, max_price):
    # Calculate the size to which sub-combinations must be generated
    min_size = len(target_key) - exploration_depth
    if min_size < 1:
        min_size = 1  # Ensuring that we don't go below size 1 which doesn't make sense

    # Generate all sub-combinations from the target key down to min_size
    sub_combinations = set()
    for size in range(len(target_key), min_size - 1, -1):
        for combo in itertools.combinations(target_key, size):
            sub_combinations.add(combo)

    # Convert list of keys into a set of tuples for faster checking
    keys_set = set(map(tuple, keys))

    # Find indices to keep (those not in sub_combinations and below max_price)
    keep_indices = []
    for index, key in enumerate(keys):
        if tuple(key) not in sub_combinations and data[1, index] <= max_price:
            keep_indices.append(index)

    # Filter the data array to keep only the selected indices
    new_data = data[:, keep_indices]

    # Filter the keys list similarly
    new_keys = [keys[i] for i in keep_indices]

    return new_data, new_keys       


In [ ]:

def process_task(B, R, N, K, U, risk, method, keys, keys_K, keys_U, gamma, runs, epsilon):
    
    probabilistic_pricebased_step = np.zeros((runs))
    vol_bandit_step = np.zeros((runs))
    ent_bandit_step = np.zeros((runs))
    rep_bandit_step = np.zeros((runs))
    blind_step = np.zeros((runs))
    epsilon_step = np.zeros((runs))
    
    probabilistic_pricebased_count = np.zeros((runs))
    vol_count = np.zeros((runs))
    ent_count = np.zeros((runs))
    rep_count = np.zeros((runs))
    blind_count = np.zeros((runs))
    epsilon_count = np.zeros((runs))
    
    complete = np.hstack((K,U))
    valid_indices = complete[0][complete[1]<B]
    offline = valid_indices.max()
    
    size_of_U = U.shape[1]  
    size_of_K = K.shape[1]
    
    greedy_performance, greedy_count = greedy(K,U,B,R)
    inv_greedy_performance, inv_greedy_count = inv_greedy(K,U,B,R)
    
    buyer_insight1_performance, buyer_insight1_count, X, price, best_key = knapsack_y(K, U, B, R, keys, keys_K, keys_U, gamma)

    U_left, keys_u_left = filter_combinations(U, keys_U, best_key, X, (B - (price + R * buyer_insight1_count)))
                                              
    while U_left.shape[1] > 0:                                         
        buyer_insight2_performance, buyer_insight2_count, X, new_price, best_key = knapsack_y(K, U_left, (B - (price + R * buyer_insight1_count)), R, keys, keys_K, keys_u_left, gamma)
            
        if buyer_insight2_count == None:
            break
        else:        
            buyer_insight1_count += buyer_insight2_count
                                              
        if buyer_insight2_performance > buyer_insight1_performance:
            buyer_insight1_performance =buyer_insight2_performance
            price = new_price
            
        U_left, keys_u_left = filter_combinations(U_left, keys_u_left, best_key, X, (B - (price + R * buyer_insight1_count)))

        
    for k in range(runs):
        probabilistic_pricebased_step[k], probabilistic_pricebased_count[k] = probabilistic_pricebased(K,U,B,R,risk)
        blind_step[k], blind_count[k] = blind_buyer(K,U,B,R)
        epsilon_step[k], epsilon_count[k] = epsilon_greedy_bandit(K,U,B,R,epsilon)
            
    probabilistic_pricebased_performance = np.mean(probabilistic_pricebased_step)
    blind_performance = np.mean(blind_step)
    epsilon_performance = np.mean(epsilon_step)  
    
    return offline, greedy_performance, inv_greedy_performance, probabilistic_pricebased_performance, blind_performance, epsilon_performance, buyer_insight1_performance, 
        greedy_count, inv_greedy_count, buyer_insight1_count, np.mean(probabilistic_pricebased_count), np.mean(blind_count), np.mean(epsilon_count)
    

In [ ]:
# Choose the parameters of the test run

correlation = 1 # Set the price-quality correlation value for the test if using correlated pricing
iterations = 100 #Set how many iterations you want to run and averag for your results
R_values = np.linspace(0.001, 0.2, 40) # Set the range of revelation values you want to test (as % of B)
size_of_V = 10 # Set the number of available individual datasets in V
size_of_K = 3 # Set the number of datasets revealed for free in the known set K
catalogue_size = 1023 # size of the catalog
price_per_image = 1 # unit price for the datasets
method = "random" # Pricing method

with open(f"MNIST_FM.json", 'r') as f:
    accuracies = np.array(json.load(f))
with open(f"MNIST_len.json", 'r') as f:
    images_per_seller = np.array(json.load(f))

In [ ]:
#Declare the empty result arrays
offline = np.zeros((len(R_values), iterations))
probabilistic_pricebased_performance = np.zeros((len(R_values), iterations))
greedy_performance = np.zeros((len(R_values), iterations))
inv_greedy_performance = np.zeros((len(R_values), iterations))
blind_performance = np.zeros((len(R_values), iterations))
epsilon_performance = np.zeros((len(R_values), iterations))
buyer_insight_performance = np.zeros((len(R_values), iterations))
    
#Declare the empty exploration count arrays
probabilistic_pricebased_counts = np.zeros((len(R_values), iterations))
greedy_counts = np.zeros((len(R_values), iterations))
inv_greedy_counts = np.zeros((len(R_values), iterations))
blind_counts = np.zeros((len(R_values), iterations))
epsilon_counts = np.zeros((len(R_values), iterations))

with ProcessPoolExecutor() as executor:
            futures = {}
            for y in range(iterations):
                full_prices = generate_prices(accuracies[0,:], method, price_per_image, correlation, len(accuracies[0,:]), size_of_V, images_per_seller)
                original = np.vstack((accuracies[0,:],full_prices))
                K, U, keys_K, keys_U = create_U_and_K_sets(original, size_of_K, keys)
                K, U, keys_K, keys_U, B = scale_sets(K, U, budget_scale, keys_K, keys_U)
                for x, R in enumerate(R_values):
                    R = R * B
                    future = executor.submit(process_task, B, R, N, K, U, risk, method, keys, keys_K, keys_U, gamma, runs, epsilon)
                    futures[future] = (x,y)
                
                
            for future in as_completed(futures):
                x, y = futures[future]
                result = future.result()
                offline[x,y] = result[0]
                greedy_performance[x,y] = result[1]
                inv_greedy_performance[x,y] = result[2]
                probabilistic_pricebased_performance[x,y] = result[3]
                blind_performance[x,y] = result[4]
                epsilon_performance[x,y] = result[5]
                buyer_insight1_performance[x,y] = result[6]

                
                greedy_counts[x,y] = result[7]
                inv_greedy_counts[x,y] = result[8]
                buyer_insight1_counts[x,y] = result[9]
                probabilistic_pricebased_counts[x,y] = result[10]
                blind_counts[x,y] = result[11]
                epsilon_counts[x,y] = result[12]